In [ ]:
%load_ext autoreload
%autoreload
from tune_and_draw import TuneAndDraw
from fedhybrid_innerloops import FedHybrid
from parameter import TuneParam,TrainParam
from costFunc import LogisticCostFunc
import numpy as np
import random
from sklearn.datasets import load_boston
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd
import numpy as np
import random
from scipy.optimize import minimize

In [ ]:
def saved_data(nclient):
    with open('setup_3_result/train_data0'+str(nclient)+'.json', 'r') as fp: # data from xiaochun code
        train_data = json.load(fp)

        
    df_train = pd.DataFrame.from_dict(train_data['user_data'])
    df_train = df_train.T

    A_train = df_train['x'].to_numpy()
    y_train = df_train['y'].to_numpy()

    for i in range(nclient):
        A_train[i]= np.array(A_train[i])
        y_train[i] = np.array(y_train[i])

    return A_train, y_train

In [ ]:
'''
for each device k, we generate samples (Xk, Yk) according to the model y = argmax(softmax(W x + b)), 
x ∈R60, W ∈ R10×60, b ∈ R10. We model Wk ∼ N (uk, 1),
bk ∼ N (uk, 1), uk ∼ N (0, α); xk ∼ N (vk, Σ), where the
covariance matrix Σ is diagonal with Σj,j = j^−1.2.
Each element in the mean vector vk is drawn from N (Bk, 1), Bk ∼N(0, β).
Therefore, α controls how much local models differ from each other and β controls how much the local data
at each device differs from that of other devices. 
We vary α, β to generate three heterogeneous distributed datasets,
denoted Synthetic (α, β), as shown in Figure 2. We also
generate one IID dataset by setting the same W, b on all
devices and setting Xk to follow the same distribution. Our
goal is to learn a global W and b.
'''
class SampleParam: 
    def __init__(self, mean, sigma):
        self.mean = mean
        self.sigma = sigma


def softmax(z):
    ex = np.exp(z)
    sum_ex = np.sum(np.exp(z))
    return ex/sum_ex       

def dataset(nclient,n):    
    nsample_param=SampleParam(4,2)
    prior_param=SampleParam(0,0.5)
    ndim=12
    nclass=2
    print('local datasizes')
    np.random.seed(2022)
    random.seed(2022)
    
    #ni = np.random.lognormal(nsample_param.mean, nsample_param.sigma, nclient).astype(int) + 50 # number of samples per client
    if nclient == 8 and n == 354:
        ni=[45, 38, 29, 12, 62, 76, 36, 56]
    else:
        p = np.zeros(nclient)
        s = 0
        for i in range(nclient):
            p[i] = random.uniform(0.1, 1)
            s += p[i]
        p = p/s
        print(p)
        ni = p * n
        ni = [int(num) for num in ni]
        ni[-1] = ni[-1] + n - sum(ni)
     
    
    
    A0 = [[] for _ in range(nclient)]
    y0 = [[] for _ in range(nclient)]
        #### define some eprior ####

    mean_W = np.random.normal(prior_param.mean, prior_param.sigma, nclient)
    print(prior_param.sigma)
    mean_b = mean_W
    mean_x = np.random.normal(prior_param.mean, prior_param.sigma, nclient)
    mean_x = np.zeros((nclient, ndim))
    for i in range(nclient):
        mean_x[i] = np.random.normal(mean_x[i], 1, ndim)
    diagonal = np.zeros(ndim)
    for j in range(ndim):
        diagonal[j] = np.power((j+1), -1.2)
    cov_x = np.diag(diagonal)
    for i in range(nclient):
        W = np.random.normal(mean_W[i], 1, (ndim, nclass))
        b = np.random.normal(mean_b[i], 1,  nclass)
        Ai = np.random.multivariate_normal(mean_x[i], cov_x, ni[i])
        yi = np.zeros(ni[i])
        for j in range(ni[i]):
            tmp = np.dot(Ai[j], W) + b
            yi[j] = np.argmax(softmax(tmp))
        A0[i] = Ai.tolist()
        y0[i] = yi.tolist()
#print('dsfadf',y0[5])
    train_data = {'users': [], 'user_data':{}, 'num_samples':[]}

    for i in range(nclient):
        uname = 'f_{0:05d}'.format(i)        
        combined = list(zip(A0[i], y0[i]))
        random.shuffle(combined)
        A0[i][:], y0[i][:] = zip(*combined) # The * operator can be used in conjunction with zip() to unzip the list.
        num_samples = len(A0[i])
        train_data['users'].append(uname) 
        train_data['user_data'][uname] = {'x': A0[i], 'y': y0[i]}
        train_data['num_samples'].append(num_samples)
        with open('setup_3_result/train_data0'+str(nclient)+'_'+str(n)+'.json', 'w') as fp:
            json.dump(train_data, fp)
        print(num_samples)


In [ ]:
def prepare(nclient):

    #dataset(nclient,n)
    A, y =saved_data(nclient)
    gamma = 1
    func = LogisticCostFunc(A, y, gamma)
    initial_x =np.array( [[0.99846005],
    [0.14017568],
    [0.43901654],
    [0.71784581],
    [0.83952553],
    [0.00799748],
    [0.7440322 ],
    [0.78949782],
    [0.12399082],
    [0.85932448],
    [0.21513528],
    [0.91062862]])
    fn_min = minimize(func.global_func, initial_x, tol=1e-30) #result object
    fn_star = fn_min.fun
    method = FedHybrid(func, fn_star)
#ns=[0,2,4,6,8]
#innerIterations = 3
    tuneAndDraw = TuneAndDraw(nclient, initial_x,setup=3)
    return method,tuneAndDraw
#tuneAndDraw.draw(method,ns,innerIterations,method.train_innerloop_lambdaBefore)


node 8, n = 354, synchronous synthetic logistic regreesion

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations = 1
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/3 synchronous synthetic logistic regreesion before.pdf"
tuneAndDraw.draw(df,method.train_innerloop_lambdaBefore,"synchronous synthetic logistic regreesion",filename,save = 0,asyn=0,possionBeta=[])

async

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations = 0 #only to point to a set of stepsizes with no real meaning
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./analyze data/analyze data/sync_and_async/3 asynchronous synthetic logistic regreesion.pdf"
tuneAndDraw.draw(df,method.asynchronousTrain2,"node 8 async synthetic logistic regreesion possionBeta=[4, 5, 10, 4, 2, 1, 2, 7, 100, 5]",filename,save=0,asyn=1,possionBeta = [4, 5, 10, 1, 2, 7,100,5])

with innerloops 4:

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations = 4
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/3 innerloop 4 synthetic logistic regreesion before2.pdf"
tuneAndDraw.draw(df,method.train_innerloop_lambdaBefore,"synthetic logistic regreesion innerloop 4",filename,save =0,asyn=0,possionBeta=[])